In [ ]:
# Python imports
import json
import os
import shutil
import warnings

# Libs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Locals
from oggm import cfg, utils, tasks
from oggm.workflow import execute_entity_task, init_glacier_directories
from oggm.core.massbalance import (ConstantMassBalance, PastMassBalance,
                                   MultipleFlowlineMassBalance)

In [ ]:
# Module logger
import logging
log = logging.getLogger(__name__)

In [ ]:
# Papermill parameters
rgi_version = '62'
baseline_climate = 'CERA+ERA5'
fl_type = 'elev_bands'
climate_qc_months = 3
border = 10
prcp_scaling_factor = 1.6

In [ ]:
rgi_version = str(rgi_version)
climate_qc_months = int(climate_qc_months)
border = int(border)
prcp_scaling_factor = float(prcp_scaling_factor)

In [ ]:
print(f'''# User params:
rgi_version = {rgi_version}
baseline_climate = {baseline_climate}
fl_type = {fl_type}
climate_qc_months = {climate_qc_months}
border = {border}
prcp_scaling_factor = {prcp_scaling_factor}
''')

In [ ]:
run_dir = utils.gettempdir('mb_calib', home=True)
dirname = os.path.join(f'RGIV{rgi_version}', f'{baseline_climate}', f'{fl_type}', f'qc{climate_qc_months}', f'pcp{prcp_scaling_factor:.1f}')
WORKING_DIR = os.path.join(run_dir, dirname)
print(f'Working dir: {WORKING_DIR}')

In [ ]:
# Start from zero?
reset = True

# Initialize OGGM and set up the run parameters
cfg.initialize(logging_level='WARNING')

# Local paths (where to write the OGGM run output)
utils.mkdir(WORKING_DIR, reset=reset);
utils.mkdir(os.path.join(WORKING_DIR, 'log'), reset=True);

SCORES_DIR = os.path.join(WORKING_DIR, '_scores')
utils.mkdir(SCORES_DIR);

In [ ]:
cfg.PATHS['working_dir'] = WORKING_DIR
cfg.PARAMS['run_mb_calibration'] = True
cfg.PARAMS['use_multiprocessing'] = True
cfg.PARAMS['continue_on_error'] = False
cfg.PARAMS['border'] = border
cfg.PARAMS['climate_qc_months'] = climate_qc_months
cfg.PARAMS['dl_verify'] = False
cfg.PARAMS['baseline_climate'] = baseline_climate
cfg.PARAMS['prcp_scaling_factor'] = prcp_scaling_factor

In [ ]:
# Prepare the preprocessed dirs
df, _ = utils.get_wgms_files()
rids = df['RGI{}0_ID'.format(rgi_version[0])]
print('Number of potential ref glaciers: {}'.format(len(rids)))

In [ ]:
base_url = f'https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.4/L1-L2_files/{fl_type}/'

print('base url: {}'.format(base_url))

In [ ]:
all_gdirs = init_glacier_directories(rids, from_prepro_level=2, prepro_base_url=base_url, prepro_rgi_version=rgi_version)

In [ ]:
ref_list_f = os.path.join(SCORES_DIR, 'available_ref_glaciers.csv')
if not os.path.exists(ref_list_f):

    # Get the reference glacier ids
    df, _ = utils.get_wgms_files()
    rids = df['RGI{}0_ID'.format(rgi_version[0])]

    if 'CRU' in baseline_climate:
        # For CRU we can't do Antarctica
        gdirs = [gdir for gdir in all_gdirs if gdir.rgi_region != '19']
    else:
        # take all
        gdirs = [gdir for gdir in all_gdirs]

    # We need to know which period we have data for
    log.workflow('Process the climate data...')
    execute_entity_task(tasks.process_climate_data, gdirs,
                        print_log=False)
    if climate_qc_months > 0:
        execute_entity_task(tasks.historical_climate_qc, gdirs,
                            print_log=False)

    # Let OGGM decide which of these have enough data
    gdirs = utils.get_ref_mb_glaciers(gdirs)

    # Manual removal of bad actors
    if 'CRU' in baseline_climate:
        bad = ['RGI60-15.06557', 'RGI60-15.06777']
        gdirs = [gd for gd in gdirs if gd.rgi_id not in bad]

    # Save the list of glaciers for later
    log.workflow('N reference glaciers after climate check: {}'.format(len(gdirs)))
    rgidf = pd.Series(data=[g.rgi_id for g in gdirs])
    rgidf.to_csv(ref_list_f, header=False)
else:
    # Read the rgi ids of the reference glaciers
    rids = pd.read_csv(ref_list_f, index_col=0, squeeze=True).values
    gdirs = [gdir for gdir in all_gdirs if gdir.rgi_id in rids]

In [ ]:
# Climate tasks
tasks.compute_ref_t_stars(gdirs)
execute_entity_task(tasks.local_t_star, gdirs)
execute_entity_task(tasks.mu_star_calibration, gdirs)

# We store the associated params
mb_calib = gdirs[0].get_climate_info()['mb_calib_params']
params_file = os.path.join(WORKING_DIR, 'ref_tstars_params.json')
with open(params_file, 'w') as fp:
    json.dump(mb_calib, fp)

# And also some statistics
diag_path = os.path.join(SCORES_DIR, 'glacier_statistics.csv')
utils.compile_glacier_statistics(gdirs, path=diag_path)
diag_path = os.path.join(SCORES_DIR, 'fixed_geometry_mass_balance.csv')
utils.compile_fixed_geometry_mass_balance(gdirs, path=diag_path)

# Tests: for all glaciers, the mass-balance around tstar and the
# bias with observation should be approx 0
log.workflow('Starting validation loop...')
for gd in gdirs:
    mbmod = MultipleFlowlineMassBalance(gd,
                                        mb_model_class=ConstantMassBalance,
                                        use_inversion_flowlines=True,
                                        bias=0)  # bias=0 because of calib!

    mb = mbmod.get_specific_mb()
    np.testing.assert_allclose(mb, 0, atol=15)  # atol for numerical errors

    mbmod = MultipleFlowlineMassBalance(gd, mb_model_class=PastMassBalance,
                                        use_inversion_flowlines=True)

    refmb = gd.get_ref_mb_data().copy()
    refmb['OGGM'] = mbmod.get_specific_mb(year=refmb.index)
    np.testing.assert_allclose(refmb.OGGM.mean(),
                               refmb.ANNUAL_BALANCE.mean(),
                               atol=15)  # atol for numerical errors

# Log
log.workflow('Calibration is done!')

## Cross-val 

In [ ]:
# Cross-validation
ref_df = pd.read_csv(os.path.join(WORKING_DIR, 'ref_tstars.csv'), index_col=0)
log.workflow('Cross-validation loop...')
for i, gdir in enumerate(gdirs):
    # Now recalibrate the model blindly
    tmp_ref_df = ref_df.loc[ref_df.index != gdir.rgi_id]
    tasks.local_t_star(gdir, ref_df=tmp_ref_df)
    tasks.mu_star_calibration(gdir)

    # Mass-balance model with cross-validated parameters instead
    mb_mod = MultipleFlowlineMassBalance(gdir,
                                         mb_model_class=PastMassBalance,
                                         use_inversion_flowlines=True)

    # Mass-balance timeseries, observed and simulated
    refmb = gdir.get_ref_mb_data().copy()
    refmb['OGGM'] = mb_mod.get_specific_mb(year=refmb.index)

    # Compare their standard deviation
    std_ref = refmb.ANNUAL_BALANCE.std()

    with warnings.catch_warnings():
        # This can trigger a divide by zero Warning
        warnings.filterwarnings("ignore", category=RuntimeWarning)
        rcor = np.corrcoef(refmb.OGGM, refmb.ANNUAL_BALANCE)[0, 1]
        if std_ref == 0:
            # such a thing happens with some geodetic values
            std_ref = refmb.OGGM.std()
            rcor = 1

        # Store the scores
        ref_df.loc[gdir.rgi_id, 'CV_MB_BIAS'] = (refmb.OGGM.mean() -
                                                 refmb.ANNUAL_BALANCE.mean())
        ref_df.loc[gdir.rgi_id, 'CV_MB_SIGMA_BIAS'] = refmb.OGGM.std() / std_ref
        ref_df.loc[gdir.rgi_id, 'CV_MB_COR'] = rcor

# Write out
ref_df.to_csv(os.path.join(SCORES_DIR, 'crossval_tstars.csv'))

scores = 'N = {}\n'.format(len(gdirs))
scores += 'Median bias: {:.2f}\n'.format(ref_df['CV_MB_BIAS'].median())
scores += 'Mean bias: {:.2f}\n'.format(ref_df['CV_MB_BIAS'].mean())
scores += 'RMS: {:.2f}\n'.format(np.sqrt(np.mean(ref_df['CV_MB_BIAS']**2)))
scores += 'Sigma bias: {:.2f}\n'.format(np.mean(ref_df['CV_MB_SIGMA_BIAS']))

# Marzeion et al Figure 3
f, ax = plt.subplots(1, 1)
bins = np.arange(20) * 400 - 3800
ylim = 130
ref_df['CV_MB_BIAS'].plot(ax=ax, kind='hist', bins=bins, color='C3',
                          label='')
ax.vlines(ref_df['CV_MB_BIAS'].mean(), 0, ylim, linestyles='--',
          label='Mean')
ax.vlines(ref_df['CV_MB_BIAS'].quantile(), 0, ylim, label='Median')
ax.vlines(ref_df['CV_MB_BIAS'].quantile([0.05, 0.95]), 0, ylim,
          color='grey', label='5% and 95%\npercentiles')
ax.text(0.01, 0.99, scores,
        horizontalalignment='left',
        verticalalignment='top',
        transform=ax.transAxes)

ax.set_ylim(0, ylim)
ax.set_ylabel('N Glaciers')
ax.set_xlabel('Mass-balance error (mm w.e. yr$^{-1}$)')
ax.legend(loc='best')
plt.title('{}'.format(dirname.replace('/', ' ')))
plt.tight_layout()
plt.savefig(os.path.join(SCORES_DIR, 'cv_histogram.png'),
            dpi=150, bbox_inches='tight')
plt.show()

# Output
print(scores)
fn = os.path.join(SCORES_DIR, 'scores.txt')
with open(fn, 'w') as f:
    f.write(scores)

## Ref Tstar analysis

In [ ]:
df = pd.read_csv(os.path.join(WORKING_DIR, 'ref_tstars.csv'), index_col=0)
df.bias.plot(kind='hist');
df.bias.mean(), df.bias.median()

## Delete data

In [ ]:
shutil.rmtree(os.path.join(WORKING_DIR, 'log'))
shutil.rmtree(os.path.join(WORKING_DIR, 'per_glacier'))